In [ ]:
import numpy as np

In [ ]:
def sigmoid(x):
  return 1/(1+np.exp(-x))

def relu(x):
  return np.maximum(0,x)  

def softmax(X):
  return np.exp(X)/(np.sum(np.exp(X),axis=1).reshape(-1,1))     

In [ ]:
def affine(b,W,X):
  return b + np.matmul(X,W)

def fully_connected(b,W,activ,X):
  Z = affine(b,W,X)
  if activ == 'sigmoid':
    return sigmoid(Z)
  if activ == 'relu':
    return relu(Z)
  if activ == 'softmax':
    return softmax(Z)
  if activ == 'tanh':
    return np.tanh(Z)
  return Z    

In [ ]:
def padd(p,A):
  return np.pad(A,((0,0),(p,p),(p,p),(0,0)))

def row_conv_2d(W,s,sec_A):
  f = W.shape[1]
  ncA = sec_A.shape[1]
  row = np.array([])
  j = 0
  while j+f <= ncA:
    row = np.append(row,np.sum(sec_A[:,j:j+f]*W))
    j = j+s
  return row  

def conv_2d_2d(W,s,A):
  f = W.shape[0]
  nrA = A.shape[0]
  i = 0
  sec_A = A[i:i+f,:]
  row = row_conv_2d(W,s,sec_A)
  B = row.reshape((1,-1))
  i = i+s   
  while i+f <= nrA:
    sec_A = A[i:i+f,:]
    row = row_conv_2d(W,s,sec_A)
    row = row.reshape((1,-1))
    B = np.append(B, row, axis = 0)    
    i = i+s
  return B

def row_conv_3d(W,s,sec_A):
  f = W.shape[1]
  ncA = sec_A.shape[1]
  row = np.array([])
  j = 0
  while j+f <= ncA:
    row = np.append(row,np.sum(sec_A[:,j:j+f,:]*W))
    j = j+s
  return row  
  
def conv_3d_3d(W,s,A):
  f = W.shape[0]
  nrA = A.shape[0]
  i = 0
  sec_A = A[i:i+f,:,:]
  row = row_conv_3d(W,s,sec_A)
  B = row.reshape((1,-1))
  i = i+s   
  while i+f <= nrA:
    sec_A = A[i:i+f,:,:]
    row = row_conv_3d(W,s,sec_A)
    row = row.reshape((1,-1))
    B = np.append(B, row, axis = 0)    
    i = i+s
  return B  

def conv_4d_3d(W,s,A):
  n_filters = W.shape[3]
  filter = W[:,:,:,0]
  C = conv_3d_3d(filter,s,A)
  B = C.reshape((C.shape[0],C.shape[1],1))
  for i in range(1,n_fliters):
    filter = W[:,:,:,i]
    C = conv_3d_3d(filter,s,A)
    C = C.reshape((C.shape[0],C.shape[1],1))
    B = np.append(B, C, axis = 2)    
  return B    

def conv(W,s,A):
  n_ex = A.shape[0]
  A_ex = A[0,:,:,:]
  C = conv_4d_3d(W,s,A_ex)
  B = C.reshape((1,C.shape[0],C.shape[1],C.shape[2]))
  for i in range(1,n_ex):
    A_ex = A[i,:,:,:]
    C = conv_4d_3d(W,s,A_ex)
    C = C.reshape((1,C.shape[0],C.shape[1],C.shape[2]))
    B = np.append(B, C, axis = 0)    
  return B  

def convolutional(b,W,s,p,activ,X):
  Z = conv(W,s,padd(p,X))+b
  if activ == 'sigmoid':
    return sigmoid(Z)
  if activ == 'relu':
    return relu(Z)
  if activ == 'tanh':
    return np.tanh(Z)
  return Z    

In [ ]:
def row_pool(f,s,sec_A):
  ncA = sec_A.shape[1]
  row = np.array([])
  j = 0
  while j+f <= ncA:
    row = np.append(row,np.max(sec_A[:,j:j+f]))
    j = j+s
  return row  

def pool_2d(f,s,A):
  i = 0
  nrA = A.shape[0]
  C = row_pool(f,s,A[0:0+f,:]).reshape((1,-1))
  i = i+s
  while i + f <= nrA:
    C = np.append(C,row_pool(f,s,A[i:i+f,:]).reshape((1,-1)), axis = 0)
    i = i+s
  return C  

def pool_one_ex(f,s,A):
  B = pool_2d(f,s,A[:,:,0])
  C = B.reshape((B.shape[0],B.shape[1],1))
  for i in range(1,A.shape[2]):
    B = pool_2d(f,s,A[:,:,i])
    B = B.reshape((B.shape[0],B.shape[1],1))
    C = np.append(C,B, axis = 2)
  return C 

def pool(f,s,A):
  B = pool_one_ex(f,s,A[0,:,:,:])
  C = B.reshape((1,B.shape[0],B.shape[1],B.shape[2]))
  for i in range(1,A.shape[0]):
    B = pool_one_ex(f,s,A[i,:,:,:])
    B = B.reshape((1,B.shape[0],B.shape[1],B.shape[2]))
    C = np.append(C,B, axis = 0)
  return C  

In [ ]:
# Input data
#
# Features in X. X.shape = (n_e, n_h, n_w, n_c) n_e = number of examples
# n_c = 3 if color images. n_c = 1 if black and white image 
#
# Labels in Y. Y.shape = (n_e, n_cat) n_cat = number of categories

In [ ]:
# Type network (hyperparameters)
#
# activation = [None, '','',....... ] relu, sigmoid, tanh, softmax, lin
# layer_type = [None, '','',.....] convolutional_layer, pooling_layer, dense_layer
# shape_data = [X.shape[1:], (), (),....] shape of the data in the layer without 
# the first component, that corresponds to the number of example
# paddings = [None,,,.....] some are numbers, some are None
# strides = [None,,,.....] some are numbers, some are None
# filters_sizes = [None,,,.....] some are numbers, some are None
# L = number of layers not counting the input layer 
# L = len(shape_data) - 1

In [ ]:
def first_dense_layer(layer_type):
  l = 1
  while layer_type[l] != 'dense_layer':
    l = l+1 
  return l

first_dense = first_dense_layer(layer_type) 

NameError: ignored

In [ ]:
def data_next_layer(W,b,activ,layer_type,filter_size,stride,padding,A):
  if layer_type == 'dense_layer':
    return fully_connected(b,W,activ,A)
  if layer_type == 'convolutional_layer':
    return convolutional(b,W,stride,padding,activ,A)
  if layer_type == 'pooling_layer':
    return pool(filter_size,stride,A)

def data_in_layers(W,b,activation,layer_type,filters_sizes,strides,paddings,first_dense,X):
  A = [X]
  for l in range(1,L+1):
    A_prev = A[l-1]
    if l == first_dense:
      A_prev = A_prev.reshape((A_prev.shape[0],-1))
    A.append(data_next_layer(W[l],b[l],activation[l],layer_type[l],filters_sizes[l],strides[l],paddings[l],A_prev))
  return A

def predictions(W,b,activation,layer_type,filters_sizes,strides,paddings,first_dense,X):
  A = data_in_layers(W,b,activation,layer_type,filters_sizes,strides,paddings,first_dense,X)
  return A[-1]  